In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

# template = PromptTemplate.from_template("{country_a}에서 {country_b}까지의 거리는 얼마나 되지?")

# prompt = template.format(country_a="영국", country_b="독일")

# chat.predict(prompt)


In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "初めまして, 私の名前は {name}です。"),
        ("human", "{country_a}から{country_b}までの距離は？そしてあなたのお名前は？")
    ]
)

prompt = template.format_messages(
    language="Japanese",
    name="기가정원",
    country_a="한국",
    country_b="일본"
)
chat.predict_messages(prompt)
# from langchain.schema import HumanMessage, AIMessage, SystemMessage
 
# messages =[
#     SystemMessage(content="You are a geography expert. And you only reply in Korean"),
#     AIMessage(content="안녕하세요, 제이름은 기가정원입니다."),
#     HumanMessage(content="한국의 서울에서 일본 도쿄까지의 거리는 얼마나 되지? 그리고 너의 이름은 뭐지?"),
# ]

# chat.predict_messages(messages)


AIMessage(content='こんにちは、私の名前はギガジョンウォンです。韓国から日本までの距離は、直線距離で約900キロメートルです。')

# 3.3 OutputParser and LCEL
## OutputParser
LLM 의 응답(Response)를 변형해야할 경우 사용. (LLM은 항상 평문으로 오기때문에...)

## 이번장 목표
응답 값을 list로 변형 하기

In [1]:
from langchain.schema import BaseOutputParser

class CommaOututParser(BaseOutputParser):
    def parse(self, text):
        # strip : 텍스트 앞뒤 공백 제거
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOututParser()
p.parse(" 내 , 이름 , 정원 ")

['내', '이름', '정원']

In [10]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everthing you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

# prompt = template.format_messages(max_items=10, question="What are the colors?")

# result = chat.predict_messages(prompt)

# p = CommaOututParser()
# p.parse(result.content)

chain = template | chat | CommaOututParser()
chain.invoke({"question":"What are the pokemons?","max_items":10})

['pikachu',
 'charizard',
 'bulbasaur',
 'squirtle',
 'jigglypuff',
 'eevee',
 'snorlax',
 'mewtwo',
 'articuno',
 'zapdos']

# 3.4 Chaning Chains


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat



In [9]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "korean"})



Great choice! Korean cuisine is known for its bold flavors and unique combinations. Here's a recipe for a classic Korean dish called Bibimbap:

Ingredients:
- 2 cups cooked short-grain rice
- 1 cup thinly sliced beef (you can use sirloin or ribeye)
- 1 cup julienned carrots
- 1 cup julienned cucumber
- 1 cup bean sprouts
- 1 cup spinach
- 4 eggs
- 4 tablespoons soy sauce
- 2 tablespoons sesame oil
- 2 tablespoons gochujang (Korean chili paste)
- 2 tablespoons vegetable oil
- Salt, to taste
- Sesame seeds, for garnish

Instructions:
1. Marinate the beef: In a bowl, combine the beef, 2 tablespoons soy sauce, and 1 tablespoon sesame oil. Mix well and let it marinate for at least 15 minutes.

2. Prepare the vegetables: Blanch the bean sprouts and spinach in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach. Season both with a pinch of salt and 1 tablespoon sesame oil.

3. Cook the beef: Heat 1 tablespoon of vegetable oil in a pan 

AIMessageChunk(content="Here's a vegetarian version of the Bibimbap recipe:\n\nIngredients:\n- 2 cups cooked short-grain rice\n- 1 cup thinly sliced tofu or tempeh (instead of beef)\n- 1 cup julienned carrots\n- 1 cup julienned cucumber\n- 1 cup bean sprouts\n- 1 cup spinach\n- 4 eggs (optional, omit for a vegan version)\n- 4 tablespoons soy sauce\n- 2 tablespoons sesame oil\n- 2 tablespoons gochujang (Korean chili paste)\n- 2 tablespoons vegetable oil\n- Salt, to taste\n- Sesame seeds, for garnish\n\nInstructions:\n1. Marinate the tofu or tempeh: In a bowl, combine the tofu or tempeh with 2 tablespoons soy sauce and 1 tablespoon sesame oil. Let it marinate for at least 15 minutes.\n\n2. Prepare the vegetables: Blanch the bean sprouts and spinach in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach. Season both with a pinch of salt and 1 tablespoon sesame oil.\n\n3. Cook the tofu or tempeh: Heat 1 tablespoon of vegetable oil i